In [1]:
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
import re
import string
import operator
from nltk.util import ngrams
import numpy as np
nltk.download('punkt')
from google.colab import drive
drive.mount("/content/drive")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import os
os.listdir('drive/My Drive/Fake Image Detection/')

['x_test_r50.npy',
 'x_train_r50.npy',
 'y_test.npy',
 'y_train.npy',
 'Evaluation Model.ipynb',
 'maskrcnn',
 'test.txt',
 'train.txt']

In [0]:
# read file from google-drive, colab doesnt keeps uploaded files persistently

tweet_file = open(r"drive/My Drive/Fake Image Detection/train.txt", encoding="utf8")
tweets = tweet_file.read()

# test file separate
tweets_file_test = open(r"drive/My Drive/Fake Image Detection/test.txt", encoding="utf8")
tweets_test = tweets_file_test.read()

# tweets_list = nltk.sent_tokenize(tweets)
tweets_list = re.split('\n\n',tweets)
tweets_list_test = re.split('\n\n',tweets_test)

tweets_list = tweets_list[:-1]
tweets_list_test = tweets_list_test[:-1]

In [4]:
#clean train dataset
tweets_clean_list = []
sentiments_list = []

for sent in tweets_list:
  # remove \n
  temp = re.sub("\n", ' ', sent)

  # remove ’
  temp = re.sub("\’", '', sent)

  # convert to lowercase
  temp = temp.lower()

  # remove unnecessary punctuations
  temp = " ".join("".join([" " if ch in string.punctuation else ch for ch in temp]).split())
  
  # remove first two words from the message, its metadata
  temp = temp.split(' ', 3)

  # take the next word as sentiment
  sentiment = temp[2]

  # Retreive only the message
  new_msg = temp[3]

  new_msg = new_msg.replace("hin", "")
  new_msg = new_msg.replace("eng", "")
  new_msg = new_msg.replace("o", "")

  # emoji removal by emoji pattern
  emoji_pattern = re.compile("["
                        u"\U0001F600-\U0001F64F"  # emoticons
                        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                        u"\U0001F680-\U0001F6FF"  # transport & map symbols
                        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                        u"\U00002702-\U000027B0"
                        u"\U000024C2-\U0001F251"
                        "]+", flags=re.UNICODE)
  new_msg = emoji_pattern.sub(r'', new_msg)

  # remove multiple consecutive spaces from message
  new_msg = re.sub(' +', ' ', new_msg)

  #after cleaning up, append to new list
  tweets_clean_list.append(new_msg)
  sentiments_list.append(sentiment)

tweets_clean_list[:10]
sentiments_list[:10]

['negative',
 'negative',
 'positive',
 'positive',
 'positive',
 'negative',
 'positive',
 'negative',
 'positive',
 'negative']

In [28]:
#Clean Test dataset\
#def hello():
tweets_clean_list_test = []
sentiments_list_test = []

for sent in tweets_list_test:
  # remove \n
    temp = re.sub("\n", ' ', sent)

  # remove ’
    temp = re.sub("\’", '', sent)

  # remove unnecessary punctuations
    temp = " ".join("".join([" " if ch in string.punctuation else ch for ch in temp]).split())
  
  # remove first three words from the message, its metadata
    temp = temp.split(' ', 3)

  # take the next word as sentiment
    sentiment = temp[2]

  # Retreive only the message
    new_msg = temp[3]

    new_msg = new_msg.replace("hin", "")
    new_msg = new_msg.replace("eng", "")
    new_msg = new_msg.replace("o", "")

  # emoji removal by emoji pattern
    emoji_pattern = re.compile("["
                        u"\U0001F600-\U0001F64F"  # emoticons
                        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                        u"\U0001F680-\U0001F6FF"  # transport & map symbols
                        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                        u"\U00002702-\U000027B0"
                        u"\U000024C2-\U0001F251"
                        "]+", flags=re.UNICODE)
    new_msg = emoji_pattern.sub(r'', new_msg)

    new_msg = re.sub(' +', ' ', new_msg)

    #after cleaning up, append to new list
    tweets_clean_list_test.append(new_msg)
    sentiments_list_test.append(sentiment)

In [0]:
train_dict = {"Message": tweets_clean_list, "Sentiment": sentiments_list}
df = pd.DataFrame(train_dict)

### removing extra spaces from the tweets

In [0]:
mess = []
for message in df['Message'].values:
    x = re.sub(' +', ' ',message)
    z = x.lstrip()
    y = z.rstrip()
    mess.append(y)

In [0]:
df['Message_clean'] = mess

In [0]:
df = df.drop(['Message'],axis=1)

In [10]:
df.head()

,Sentiment,Message_clean
0,negative,adilnisarbutt pakistan ka ghra tauq he pakista...
1,negative,madarchd mulle ye mathura me nahi dikha tha ja...
2,positive,narendramdi manya pradhan mantri mahday shrima...
3,positive,atheist krishna jcb full trend me chal rahi aa
4,positive,abhisharsharma ravishkumarblg lksabha me janta...


### for test dataset

In [0]:
train_dict = {"Message": tweets_clean_list_test, "Sentiment": sentiments_list_test}
df_test = pd.DataFrame(train_dict)

In [0]:
mess = []
for message in df_test['Message'].values:
    x = re.sub(' +', ' ',message)
    z = x.lstrip()
    y = z.rstrip()
    mess.append(y)

In [0]:
df_test['Message_clean'] = mess

In [0]:
df_test = df_test.drop(['Message'],axis=1)

In [15]:
df_test.head()

,Sentiment,Message_clean
0,neutral,rt uaapcnfessins lve lks gd n maddie ak lang b...
1,neutral,ye ye ye we gnna start anther june n a sur nte...
2,neutral,zwfffy9jgklela1 min f lycg thakurdadu089 manak...
3,negative,caring bht jyada caring curier wale bsdk ke si...
4,positive,alihzaidipti sarfaraza 54 what nnesense kabhi ...


In [16]:
stopwords = set([])
len(tweets_clean_list_test),len(sentiments_list_test)

(1869, 1869)

In [0]:
# https://stackoverflow.com/a/47091490/4084039
import re

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [18]:
# Combining all the above stundents 
from tqdm import tqdm
from bs4 import BeautifulSoup
preprocessed_text = []
# tqdm is for printing the status bar
for sentance in tqdm(df['Message_clean'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_text.append(sentance.strip())

100%|██████████| 15131/15131 [00:04<00:00, 3135.16it/s]


In [0]:
df['Message'] = preprocessed_text

In [0]:
pre = preprocessed_text
preprocessed_text = []
for val in pre:
    x = ""
    for w in val.split(' '):
        if len(w)>1:
        x = x+" "+w
    preprocessed_text.append(x)

In [0]:
df['single'] = preprocessed_text

In [22]:
# Combining all the above stundents 
from tqdm import tqdm
from bs4 import BeautifulSoup
preprocessed_text = []
# tqdm is for printing the status bar
for sentance in tqdm(df_test['Message_clean'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_text.append(sentance.strip())

100%|██████████| 1869/1869 [00:00<00:00, 3011.82it/s]


In [0]:
df_test['Message'] = preprocessed_text

## Removing all the words with single length

In [0]:
pre = preprocessed_text
preprocessed_text = []
for val in pre:
  #print(val)
    x = ""
    for w in val.split(' '):
        if len(w)>1:
        x = x+" "+w
  #print(x)
  #break
    preprocessed_text.append(x)

In [0]:
df_test['single'] = preprocessed_text

In [26]:
## test data
df_test.head()

,Sentiment,Message_clean,Message,single
0,neutral,rt uaapcnfessins lve lks gd n maddie ak lang b...,rt uaapcnfessins lve lks gd n maddie ak lang b...,rt uaapcnfessins lve lks gd maddie ak lang ba...
1,neutral,ye ye ye we gnna start anther june n a sur nte...,ye ye ye we gnna start anther june n a sur nte...,ye ye ye we gnna start anther june sur nte uh...
2,neutral,zwfffy9jgklela1 min f lycg thakurdadu089 manak...,min f lycg manakgupta fficefknath mein kahna n...,min lycg manakgupta fficefknath mein kahna na...
3,negative,caring bht jyada caring curier wale bsdk ke si...,caring bht jyada caring curier wale bsdk ke si...,caring bht jyada caring curier wale bsdk ke s...
4,positive,alihzaidipti sarfaraza 54 what nnesense kabhi ...,alihzaidipti sarfaraza what nnesense kabhi bay...,alihzaidipti sarfaraza what nnesense kabhi ba...


In [27]:
## train data
df.head()

,Sentiment,Message_clean,Message,single
0,negative,adilnisarbutt pakistan ka ghra tauq he pakista...,adilnisarbutt pakistan ka ghra tauq he pakista...,adilnisarbutt pakistan ka ghra tauq he pakist...
1,negative,madarchd mulle ye mathura me nahi dikha tha ja...,madarchd mulle ye mathura me nahi dikha tha ja...,madarchd mulle ye mathura me nahi dikha tha j...
2,positive,narendramdi manya pradhan mantri mahday shrima...,narendramdi manya pradhan mantri mahday shrima...,narendramdi manya pradhan mantri mahday shrim...
3,positive,atheist krishna jcb full trend me chal rahi aa,atheist krishna jcb full trend me chal rahi aa,atheist krishna jcb full trend me chal rahi aa
4,positive,abhisharsharma ravishkumarblg lksabha me janta...,abhisharsharma ravishkumarblg lksabha me janta...,abhisharsharma ravishkumarblg lksabha me jant...


#### Applying LSTM 

In [0]:
words = []
for k in df['single']:
    for word in k.split(' '):
        if word not in words:
            words.append(word) 

In [0]:
all_tweets = []
for k in df['single']:
    for word in k.split(' '):
        all_tweets.append(word)

In [30]:
print("Total number of unique words in thhe dataset: ",len(words))
print("Total words in the dataset: ",len(all_tweets))

Total number of unique words in thhe dataset:  37504
Total words in the dataset:  268258


In [0]:
# initializing the dictionary with all unique values
dictionary = {}
for k in all_tweets:
        dictionary[k] = 0 

In [0]:
for k in all_tweets:
    dictionary[k] = dictionary[k]+1   # creating the dictionary from the given reviews

In [0]:
import operator
sorted_dict = sorted(dictionary.items(), key=operator.itemgetter(1))  # sorting the dictionary

In [34]:
sd = sorted_dict[::-1]   # containing all the words count in dec order
sd[:10]

[('', 15131),
 ('ki', 4824),
 ('hai', 4464),
 ('nahi', 2872),
 ('ka', 2575),
 ('ke', 2467),
 ('rt', 2390),
 ('yu', 2284),
 ('me', 2179),
 ('se', 1987)]

In [35]:
#sd = sorted_dict[: :-1]
print("TOP WORDS\t\t\t"+"FREQUENCY")
count = 0
for d in sd:
    if count ==10:
        break
    print(d[0],"\t\t\t\t",d[1])
    count = count+1  

TOP WORDS			FREQUENCY
 				 15131
ki 				 4824
hai 				 4464
nahi 				 2872
ka 				 2575
ke 				 2467
rt 				 2390
yu 				 2284
me 				 2179
se 				 1987


In [0]:
final_words = []
for word in sd:
    final_words.append(word[0])

In [0]:
## convertiing all the training set into numerical representation
new_tweets = []   # it will contain the numerical representation of the tweets
for f in df['single']:
    l = []
    for k in f.split(' '):
        if k in final_words:
            ind = final_words.index(k)
            l.append(ind+1)
    new_tweets.append(l)

In [0]:
## convertiing all the test set into numerical representation
test_tweets = []   # it will contain the numerical representation of the tweets
for f in df_test['single']:
    l = []
    for k in f.split(' '):
        if k in final_words:
            ind = final_words.index(k)
            l.append(ind+1)
    test_tweets.append(l)

In [0]:
x_train= new_tweets
x_test=test_tweets

In [40]:
# padding the dataset
from keras.preprocessing import sequence
max_review_length = 600
x_train=sequence.pad_sequences(x_train,maxlen=max_review_length)
x_test=sequence.pad_sequences(x_test,maxlen=max_review_length)
print("Training data Shape",x_train.shape)

Using TensorFlow backend.


Training data Shape (15131, 600)


In [0]:
y_test1 = df_test['Sentiment']
y_test = []
for val in y_test1:
    if val=='neutral':
        y_test.append(1)
    if val=='positive':
        y_test.append(2)
    if val=='negative':
        y_test.append(3)

In [42]:
df.head()

,Sentiment,Message_clean,Message,single
0,negative,adilnisarbutt pakistan ka ghra tauq he pakista...,adilnisarbutt pakistan ka ghra tauq he pakista...,adilnisarbutt pakistan ka ghra tauq he pakist...
1,negative,madarchd mulle ye mathura me nahi dikha tha ja...,madarchd mulle ye mathura me nahi dikha tha ja...,madarchd mulle ye mathura me nahi dikha tha j...
2,positive,narendramdi manya pradhan mantri mahday shrima...,narendramdi manya pradhan mantri mahday shrima...,narendramdi manya pradhan mantri mahday shrim...
3,positive,atheist krishna jcb full trend me chal rahi aa,atheist krishna jcb full trend me chal rahi aa,atheist krishna jcb full trend me chal rahi aa
4,positive,abhisharsharma ravishkumarblg lksabha me janta...,abhisharsharma ravishkumarblg lksabha me janta...,abhisharsharma ravishkumarblg lksabha me jant...


In [0]:
y_test1 = df['Sentiment']
y_train = []
for val in y_test1:
    if val=='neutral':
        y_train.append(1)
    if val=='positive':
        y_train.append(2)
    if val=='negative':
        y_train.append(3)

In [44]:
len(y_train),len(x_train)

(15131, 15131)

In [0]:
# Reference: https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/
# LSTM for sequence classification in the IMDB dataset
import tensorflow as tf
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dense,Dropout
from keras.layers.embeddings import Embedding
# fix random seed for reproducibility
numpy.random.seed(7)
#from tensorflow.python.framework import ops
#ops.reset_default_graph()

In [46]:
# create the model
embedding_vecor_length = 32
top_words = 37529
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=len(x_train[0])))
model.add(LSTM(32,return_sequences=True))
model.add(Dropout(.5))
model.add(LSTM(64,return_sequences=True))
model.add(Dropout(0.75))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
#Refer: https://datascience.stackexchange.com/questions/10615/number-of-parameters-in-an-lstm-model





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 600, 32)           1200928   
_________________________________________________________________
lstm_1 (LSTM)                (None, 600, 32)           8320      
_________________________________________________________________
dropout_1 (Dropout)          (None, 600, 32)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 600, 64)           24832     
_________________________________________________________________
dropout_2 (Dropout)          (None, 600, 64)           0         
_____________________________________

In [5]:
history = model.fit(x_train, y_train, nb_epoch=10, batch_size=64,validation_split=.1)




Train on 13617 samples, validate on 1514 samples
Epoch 1/10





13617/13617 [==============================] - 729s 54ms/step - loss: -9.1801 - acc: 0.3762 - val_loss: -16.1870 - val_acc: 0.3263
Epoch 2/10
13617/13617 [==============================] - 719s 53ms/step - loss: -14.3272 - acc: 0.3812 - val_loss: -17.3850 - val_acc: 0.3305
Epoch 3/10
13617/13617 [==============================] - 720s 53ms/step - loss: -14.4017 - acc: 0.3917 - val_loss: -17.3850 - val_acc: 0.3642
Epoch 4/10
13617/13617 [==============================] - 717s 53ms/step - loss: -14.4017 - acc: 0.4247 - val_loss: -17.3850 - val_acc: 0.3524
Epoch 5/10
13617/13617 [==============================] - 720s 53ms/step - loss: -14.4017 - acc: 0.4126 - val_loss: -17.3850 - val_acc: 0.3791
Epoch 6/10
13617/13617 [==============================] - 718s 53ms/step - loss: -14.4017 - acc: 0.4362 - val_loss: -17.3850 - val_acc: 0.4127
Epoch 7/10
13617/13617 [==============================] - 725s 53ms/step - loss: -14.4

### Report of the Model

In [11]:
from sklearn.metrics import classification_report as report
y_pre = model.predict_classes(X_test)
result = report(y_test,y_pre).split("\n")
for d in result:
    print(d)

              precision    recall  f1-score   support

           1       0.52      0.59      0.54       754
           2       0.54      0.52      0.58       582
           3       0.61      0.41      0.49       533

    accuracy                           0.45      1869
   macro avg       0.56      0.53      0.53      1869
weighted avg       0.52      0.52      0.51      1869


### taking top 5000 words for the modeling

In [13]:
# tsking top 5000 points
top_5000 = []
count = 0
for d in sd:
    if count ==5000:
        break
    top_5000.append(d)
    count = count+1

In [0]:
top_5000[0:11]

In [0]:
final_words = []
for word in top_5000:
    final_words.append(word[0])

In [0]:
new_tweets = []   # it will contain the numerical representation of the reviews
for f in df['single']:
    l = []
    for k in f.split(' '):
        if k in final_words:
            ind = final_words.index(k)
            l.append(ind+1)
    new_tweets.append(l)

In [0]:
test_tweets = []   # it will contain the numerical representation of the reviews
for f in df_test['single']:
    l = []
    for k in f.split(' '):
        if k in final_words:
            ind = final_words.index(k)
            l.append(ind+1)
    test_tweets.append(l)

In [0]:
# padding the dataset
max_review_length = 600
x_train=sequence.pad_sequences(x_train,maxlen=max_review_length)
x_test=sequence.pad_sequences(x_test,maxlen=max_review_length)

In [0]:
# create the model
embedding_vecor_length = 32
top_words = 37529
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=len(x_train[0])))
model.add(LSTM(32,return_sequences=True))
model.add(Dropout(.5))
model.add(LSTM(64,return_sequences=True))
model.add(Dropout(0.75))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
#Refer: https://datascience.stackexchange.com/questions/10615/number-of-parameters-in-an-lstm-model

In [14]:
history = model.fit(x_train, y_train, nb_epoch=10, batch_size=64,validation_split=.1)




Train on 13617 samples, validate on 1514 samples
Epoch 1/10





13617/13617 [==============================] - 729s 54ms/step - loss: -9.1801 - acc: 0.3762 - val_loss: -16.1870 - val_acc: 0.3305
Epoch 2/10
13617/13617 [==============================] - 719s 53ms/step - loss: -14.3272 - acc: 0.3928 - val_loss: -17.3850 - val_acc: 0.3571
Epoch 3/10
13617/13617 [==============================] - 720s 53ms/step - loss: -14.4017 - acc: 0.4176 - val_loss: -17.3850 - val_acc: 0.3721
Epoch 4/10
13617/13617 [==============================] - 717s 53ms/step - loss: -14.4017 - acc: 0.4409 - val_loss: -17.3850 - val_acc: 0.4120
Epoch 5/10
13617/13617 [==============================] - 720s 53ms/step - loss: -14.4017 - acc: 0.4381 - val_loss: -17.3850 - val_acc: 0.3989
Epoch 6/10
13617/13617 [==============================] - 718s 53ms/step - loss: -14.4017 - acc: 0.4721 - val_loss: -17.3850 - val_acc: 0.4265
Epoch 7/10
13617/13617 [==============================] - 725s 53ms/step - loss: -14.4

### Report of the model

In [16]:
### classical Machine Learning
from sklearn.metrics import classification_report as report
y_pre = model.predict_classes(X_test)
result = report(y_test,y_pre).split("\n")
for d in result:
    print(d)

              precision    recall  f1-score   support

           1       0.54      0.63      0.63       754
           2       0.56      0.61      0.65       582
           3       0.63      0.45      0.58       533

    accuracy                           0.47      1869
   macro avg       0.57      0.56      0.59      1869
weighted avg       0.59      0.62      0.56      1869
